![Stucture of the cvs](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)

Basic analysis about the data is done by others：https://www.kaggle.com/codename007/home-credit-complete-eda-feature-importance

In [1]:
import gc
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
%matplotlib inline

In [2]:
import os
PATH = "./csv/" 
print(os.listdir(PATH))

['application_test.csv', 'application_train.csv', 'bureau.csv', 'bureau_balance.csv', 'credit_card_balance.csv', 'installments_payments.csv', 'POS_CASH_balance.csv', 'previous_application.csv', 'sample_submission.csv']


As the data is too big (csv files are about 2.49 GB in Windows file explorer, reading in python is even larger), we need to reduce the memory usage of the data by modifying the data type.

In [3]:
#modified from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage, but not changing all the object to catagory here.
def reduce_mem(df):
    """
    modify the data type columns by columns to reduce the memory usage.
    """
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            col_min = df[col].min()
            col_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if col_max < np.iinfo(np.int8).max and col_min > np.iinfo(np.int8).min:
                    df[col] = df[col].astype(np.int8)
                elif col_max < np.iinfo(np.int16).max and col_min > np.iinfo(np.int16).min:
                    df[col] = df[col].astype(np.int16)
                elif col_max < np.iinfo(np.int32).max and col_min > np.iinfo(np.int32).min:
                    df[col] = df[col].astype(np.int32)
                elif col_max < np.iinfo(np.int64).max and col_min > np.iinfo(np.int64).min:
                    df[col] = df[col].astype(np.int64)
            else:
                if col_max < np.finfo(np.float16).max and col_min > np.finfo(np.float16).min:
                    df[col] = df[col].astype(np.float16)
                elif col_max < np.finfo(np.float32).max and col_min > np.finfo(np.float32).min:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    return df

def import_csv(file):
    df = pd.read_csv(file)
    df = reduce_mem(df)
    return df

reading the data. Using there file name as there variable name.

In [4]:
#for file in os.listdir(PATH):
#    locals()[file.split(".")[0]] = import_csv(PATH+file)
#    print(file.split(".")[0],":",locals()[file.split(".")[0]].shape[0],"rows and",locals()[file.split(".")[0]].shape[1],
#          "columns, the memory usage is {:.2f} MB".format(locals()[file.split(".")[0]].memory_usage().sum()/1024**2))

application_train.csv

application_test.csv

Combine application, using is_train variable to distinguish whether it is training set.

In [5]:
application_train = import_csv(PATH+"application_train.csv")
print("application_train: shape",application_train.shape,
      ", the memory usage is {:.2f} MB".format(application_train.memory_usage().sum()/1024**2))
application_test = import_csv(PATH+"application_test.csv")
print("application_test: shape",application_test.shape,
      ", the memory usage is {:.2f} MB".format(application_test.memory_usage().sum()/1024**2))


application_train.loc[:,"is_train"] = True 
application_test.loc[:,"is_train"] = False

data = pd.concat([application_train,application_test],axis = 0)

del application_train, application_test; gc.collect()

application_train: shape (307511, 122) , the memory usage is 92.38 MB
application_test: shape (48744, 121) , the memory usage is 14.60 MB


84

In [6]:
#factorize the catagories variables
cata = [i for i in data.columns if data[i].dtype == "object"]
for i in cata:
    data[i], index= pd.factorize(data[i])

bureau_balance.csv

In [7]:
bureau_balance = import_csv(PATH+"bureau_balance.csv")
print("bureau_balance: shape",bureau_balance.shape,", the memory usage is {:.2f} MB".format(bureau_balance.memory_usage().sum()/1024**2))

#let the status to become dummies variables
tem = pd.get_dummies(bureau_balance.STATUS, prefix='bur_bal_status')
bureau_balance = pd.concat([bureau_balance,tem],axis = 1).drop("STATUS", axis=1)

#count the SK_ID_BUREAU
bur_counts = bureau_balance[['SK_ID_BUREAU', 'MONTHS_BALANCE']].groupby('SK_ID_BUREAU').count()
bureau_balance['buro_count'] = bureau_balance['SK_ID_BUREAU'].map(bur_counts['MONTHS_BALANCE'])

#get the average of the records so that we can concat it to the main dataframe.
avg_bur_bal = bureau_balance.groupby('SK_ID_BUREAU').mean()
avg_bur_bal.columns = ['avg_bur_' + i for i in avg_bur_bal.columns]

#release the memery space
del bureau_balance,tem,bur_counts;gc.collect()

bureau_balance: shape (27299925, 3) , the memory usage is 338.46 MB


53

bureau.csv

In [8]:
bureau = import_csv(PATH+"bureau.csv")
print("bureau: shape",bureau.shape,", the memory usage is {:.2f} MB".format(bureau.memory_usage().sum()/1024**2))

#let those catagories variables to become dummies.
bur_ca_dum = pd.get_dummies(bureau.CREDIT_ACTIVE, prefix='ca_')
bur_cc_dum = pd.get_dummies(bureau.CREDIT_CURRENCY, prefix='cc_')
bur_ct_dum = pd.get_dummies(bureau.CREDIT_TYPE, prefix='ct_')
bur_full = pd.concat([bureau, bur_ca_dum, bur_cc_dum, bur_ct_dum ], axis=1).drop(["CREDIT_ACTIVE","CREDIT_CURRENCY","CREDIT_TYPE"],axis = 1)
del bur_ca_dum,bur_cc_dum,bur_ct_dum;gc.collect()

bureau: shape (1716428, 17) , the memory usage is 112.95 MB


28

Combine the information of bureau_balance.csv and bureau.csv

In [9]:
#merge bur_full and avg_bur_bal by 'SK_ID_BUREAU' as shown in the graph in the begining
bur_full = bur_full.merge(right=avg_bur_bal.reset_index(), how='left', on='SK_ID_BUREAU', suffixes=('', '_bur_bal'))

bur_per_cur = bur_full[['SK_ID_CURR', 'SK_ID_BUREAU']].groupby('SK_ID_CURR').count()
bur_full['SK_ID_BUREAU'] = bur_full['SK_ID_CURR'].map(bur_per_cur['SK_ID_BUREAU'])

avg_bur = bur_full.groupby('SK_ID_CURR').mean()

data = data.merge(right=avg_bur.reset_index(), how='left', on='SK_ID_CURR')
del bureau, bur_full,bur_per_cur,avg_bur;gc.collect()

126

POS_CASH_balance.csv

In [10]:
POS_CASH_balance = import_csv(PATH+"POS_CASH_balance.csv")
print("POS_CASH_balance: shape",POS_CASH_balance.shape,
      ", the memory usage is {:.2f} MB".format(POS_CASH_balance.memory_usage().sum()/1024**2))

#dummy the catagory variables.
POS_CASH_balance = pd.concat([POS_CASH_balance, pd.get_dummies(POS_CASH_balance["NAME_CONTRACT_STATUS"])],axis = 1).drop("NAME_CONTRACT_STATUS",axis = 1)
#count number of previous records
pre_per_cur = POS_CASH_balance[["SK_ID_CURR", "SK_ID_PREV"]].groupby("SK_ID_CURR").count()
POS_CASH_balance["SK_ID_PREV"] = POS_CASH_balance["SK_ID_CURR"].map(pre_per_cur["SK_ID_PREV"])

#get the average 
avg_pos = POS_CASH_balance.groupby("SK_ID_CURR").mean()

data = data.merge(right=avg_pos.reset_index(), how='left', on='SK_ID_CURR')
del POS_CASH_balance,pre_per_cur,avg_pos;gc.collect()

POS_CASH_balance: shape (10001358, 8) , the memory usage is 238.45 MB


80

previous_application.csv

In [11]:
previous_application = import_csv(PATH+"previous_application.csv")
print("previous_application: shape",previous_application.shape,
      ", the memory usage is {:.2f} MB".format(previous_application.memory_usage().sum()/1024**2))
#get all the catagories col and dummy them
pre_cat= [ i for i in previous_application.columns if previous_application[i].dtype == 'object']
pre_dum = pd.DataFrame()
for i in pre_cat:
    pre_dum = pd.concat([pre_dum, pd.get_dummies(previous_application[i], prefix=i).astype(np.uint8)], axis=1)
previous_application = pd.concat([previous_application,pre_dum],axis = 1).drop(pre_cat,axis = 1)
del pre_cat,pre_dum;gc.collect()

#count number of previous applications
pre_per_curr = previous_application[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
previous_application['SK_ID_PREV'] = previous_application['SK_ID_CURR'].map(pre_per_curr['SK_ID_PREV'])
avg_pre = previous_application.groupby('SK_ID_CURR').mean()

data = data.merge(right=avg_pre.reset_index(), how='left', on='SK_ID_CURR')
del previous_application,avg_pre;gc.collect()

previous_application: shape (1670214, 37) , the memory usage is 309.01 MB


67

credit_card_balance.csv

In [12]:
credit_card_balance = import_csv(PATH+"credit_card_balance.csv")
print("credit_card_balance: shape",credit_card_balance.shape,
      ", the memory usage is {:.2f} MB".format(credit_card_balance.memory_usage().sum()/1024**2))
#dummy the catagory variables.
credit_card_balance = pd.concat([credit_card_balance,
                                 pd.get_dummies(credit_card_balance["NAME_CONTRACT_STATUS"],
                                                prefix='cc_bal_status_')], axis=1).drop("NAME_CONTRACT_STATUS",axis = 1)
#count number of previous records
pre_per_cur = credit_card_balance[["SK_ID_CURR", "SK_ID_PREV"]].groupby("SK_ID_CURR").count()
credit_card_balance["SK_ID_PREV"] = credit_card_balance["SK_ID_CURR"].map(pre_per_cur["SK_ID_PREV"])
#get the average and avoid the variable name conflct.
avg_cc_bal = credit_card_balance.groupby('SK_ID_CURR').mean()
avg_cc_bal.columns = ['cc_bal_' + i for i in avg_cc_bal.columns]

data = data.merge(right=avg_cc_bal.reset_index(), how='left', on='SK_ID_CURR')
del credit_card_balance, pre_per_cur,avg_cc_bal;gc.collect()

credit_card_balance: shape (3840312, 23) , the memory usage is 289.33 MB


87

installments_payments.csv

In [13]:
installments_payments = import_csv(PATH+"installments_payments.csv")
print("installments_payments: shape",installments_payments.shape,
      ", the memory usage is {:.2f} MB".format(installments_payments.memory_usage().sum()/1024**2))

#count number of previous records
pre_per_cur = installments_payments[["SK_ID_CURR", "SK_ID_PREV"]].groupby("SK_ID_CURR").count()
installments_payments["SK_ID_PREV"] = installments_payments["SK_ID_CURR"].map(pre_per_cur["SK_ID_PREV"])

#get the average and avoid the variable name conflct.
avg_ip = installments_payments.groupby('SK_ID_CURR').mean()
avg_ip.columns = ['ip_' + i for i in avg_ip.columns]

data = data.merge(right=avg_ip.reset_index(), how='left', on='SK_ID_CURR')
del installments_payments, pre_per_cur,avg_ip;gc.collect()

installments_payments: shape (13605401, 8) , the memory usage is 311.40 MB


76

In [14]:
print("the final data: shape",data.shape,
      ", the memory usage is {:.2f} MB".format(data.memory_usage().sum()/1024**2))

the final data: shape (356255, 382) , the memory usage is 744.39 MB


In [15]:
data.to_csv('data.csv')

Reference:

https://www.kaggle.com/gemartin/load-data-reduce-memory-usage

https://www.kaggle.com/oysiyl/good-fun-with-ligthgbm/code